# Imports


In [2]:
import os

from langchain_ollama import ChatOllama

os.environ["NO_PROXY"] = "localhost,127.0.0.1"
os.environ["USER_AGENT"] = "myagent"

/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


# LLM Initialisierung mit Beispiel


In [3]:
llm = ChatOllama(
    model="gpt-oss:20b-cloud",
    base_url="http://localhost:11434",
    temperature=0.7,
)

messages = [
    ("system", "Du bist ein hilfreicher Assistent."),
    ("human", "Erkläre mir die Vorteile eines lokalen LLM."),
]

response = llm.invoke(messages)
print(response.content)

### Vorteile eines lokalen LLM (Large Language Model)

| Vorteil | Warum es wichtig ist | Praktische Beispiele |
|---------|----------------------|----------------------|
| **Datenschutz & Sicherheit** | Alle Daten bleiben auf dem eigenen Gerät oder Server – keine Übermittlung an Dritte. | Für Unternehmen, die sensible Kundendaten oder interne Strategien verarbeiten (z. B. Finanz‑, Rechts- oder Gesundheitsdaten). |
| **Geringere Latenz** | Keine Netzwerk‑Round‑Trips nötig; Antwortzeiten liegen meist im Millisekundenbereich. | Echtzeit‑Anwendungen wie Chat‑Bots in Kundensupport‑Systemen oder interaktive Spiele. |
| **Offline‑Funktionalität** | Funktioniert auch ohne Internetverbindung. | Feldstudien, mobile Geräte in abgelegenen Gebieten, militärische Einsatzfelder. |
| **Kostenkontrolle** | Einmalige Anschaffung oder Lizenz + Infrastruktur, keine laufenden API‑Kosten. | Langfristige Projekte, bei denen tausende Anfragen pro Tag entstehen. |
| **Anpassbarkeit & Fein‑Tuning** | Modell ka

# Embedding Modell Initialisierung


In [4]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="embeddinggemma:300m")

# Vektordatenbank Initialisierung

### Eine Collection für eine Website und eine für ein PDF-Dokument erstellen. Anschließend Collections anzeigen lassen.


In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

# Website laden und Text extrahieren (Website)


In [6]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("inhalt"))
loader = WebBaseLoader(
    web_paths=("https://herbartgymnasium.de/2025/11/23/forschen-staunen-entdecken-erste-mint-nacht-am-hgo/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 3539


### Text in Chunks aufteilen (Website)


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 5 sub-documents.


### Chunks in Vektordatenbank speichern (Website)


In [8]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['5d7ae847-93e2-4efa-b8d1-946783dc50d5', '2fd2a11f-3901-4585-9d6b-c04dc490dec0', '2322e849-7ea3-4440-91bd-95e11427f5ee']


# PDF laden und Text extrahieren (PDF)


In [9]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/Förderverein_HGO_Satzung.pdf")

docsPDF = loader.load()

full_text = "\n".join([d.page_content for d in docsPDF])


### Text in Chunks aufteilen (PDF)


In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splitsPDF = text_splitter.split_documents(docsPDF)

print(f"Split blog post into {len(all_splitsPDF)} sub-documents.")

Split blog post into 12 sub-documents.


### Chunks in Vektordatenbank speichern (PDF)


In [11]:
document_ids = vector_store.add_documents(documents=all_splitsPDF)

print(document_ids[:3])

['47d9291f-3179-475a-bf16-969035f88d06', '3b3c7b35-25cb-4aff-bd87-a8819c76beef', 'ddb18cb7-2104-41be-9006-20269d524003']


# LLM mit Retrieval Augmented Generation verwenden


In [12]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

### Website


In [13]:
while True:
    user_query = input("Stelle eine Frage (oder 'exit' zum Beenden): ")
    if user_query.lower() == "exit":
        break
    # RAG-Beispiel: Kontext aus der Vektordatenbank abrufen und an das LLM übergeben
    query = user_query
    # Ändere k nach Bedarf (Anzahl der relevanten Chunks)
    retrieved = vector_store.similarity_search(query, k=3)
    # Serialisiere den Kontext (achte auf Token-Limits bei großen Dokumenten)
    context_text = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved
    )
    # Baue Messages mit eingebettetem Kontext (oder sende Kontext separat, je nach Bedarf)
    messages_rag = [
        ("system", "Du bist ein hilfreicher Assistent. Nutze die folgenden Dokumentkontexte zur Beantwortung der Anfrage:\n" + context_text),
        ("human", query),
    ]
    try:
        response_rag = llm.invoke(messages_rag)
        print("--- Antwort mit Retrieval-Kontext ---")
        print(response_rag.content)
    except Exception as e:
        print("Fehler beim Aufruf des LLM mit Retrieval-Kontext:", e)

--- Antwort mit Retrieval-Kontext ---
**Was war bei der MINT‑Nacht?**  

Die erste MINT‑Nacht am Herbart‑Gymnasium (HGO) war ein voller Erfolg – ein Abend voller Experimente, Entdeckungen und echter Schulgemeinschaft.  

| Bereich | Highlights |
|---------|------------|
| **Abenteuerräume (Klassen 5 / 6)** | Drei Forschungsstationen in Biologie: Experimente zur Täuschung des Tastsinns, tierische Präparate detektivisch untersuchen und mikroskopische Beobachtung von Kleinstlebewesen. |
| **Mathematik‑Nacht** | In der Aula wurden Köpfe „zum Rauchen“ gebracht: Taschenrechner glühten, Teams tüftelten an anspruchsvollen Aufgaben, die selbst die Oberstufenschüler herausforderten. Am Ende fehlten nur noch zwei Aufgaben, um die nächste Stufe zu aktivieren. |
| **Sporthalle** | 50 Kinder nutzten den Raum für Fußball, Tisch‑ und Basketball, Bowling sowie Jonglage – ein Ort zum Austoben zwischen den Stationen. |
| **Dachterrasse – Sternstunden** | Herr Große zeigte den Kindern den Saturn durch ein

### PDF
